## Работа с графом

In [ ]:
import networkx as nx
import numpy as np
import sys
np.set_printoptions(threshold=10)

In [ ]:
# Реализация полезных функций

def incidencematrix(G):
    return nx.incidence_matrix(G)

def hodgelaplacian(G):
    B = nx.incidence_matrix(G)
    L1 = B.T @ B
    return L1

def scaledhodgelaplacian(G):
    L1 = hodgelaplacian(G)
    maxeig = np.linalg.eigvalsh(L1.todense())[-1]
    return L1 / maxeig

def aggregator(gso, N):
    assert gso.shape[0] == gso.shape[1]
    # aggregation matrix : number of shifts x [shape of gso]
    shifts = [np.eye(gso.shape[0])]
    for i in range(N-1):
        if (i % 20 == 0):
            print(i)
        shifts.append(gso @ shifts[-1])
    # combined sampling matrix : number of shifts x nodes sampled x number of nodes
    return np.array(shifts)

def generate_graph(n_elecrtodes: int):
    adj_matrix = np.ones((n_elecrtodes, n_elecrtodes)) - np.eye(n_elecrtodes)
    return nx.from_numpy_array(adj_matrix)

In [ ]:
G = generate_graph(60)
S = scaledhodgelaplacian(G)

n_shifts = 100

input_path = "../data/processed_data/"

In [ ]:
agg = aggregator(S, n_shifts)
# agg = np.load(input_path + "aggregator.npy")

In [ ]:
flows = np.load("flows.npy")

validation = np.load("validation_areas.npy")
validation = np.repeat(validation, 5)

In [ ]:
shuffle_idx = np.random.permutation(20000)
flows = flows[:, shuffle_idx]
validation = validation[shuffle_idx]

In [ ]:
X = np.transpose(agg @ flows, (2, 0, 1))

## Обучение модели

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
import tensorflow.keras as keras

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [ ]:
# function to make 1D CNN classifier
def cnn_model(layer_depths, kernel_sizes, dropout_rate, learning_rate, num_classes):
    model = keras.Sequential()

    for l, k in zip(layer_depths, kernel_sizes):
        model.add(keras.layers.Conv1D(filters=l, kernel_size=k, strides=1,
                                      padding='same', activation=keras.activations.relu))
        model.add(keras.layers.MaxPool1D(pool_size=2, strides=2, padding='same'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(rate=dropout_rate))
    model.add(keras.layers.Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=tf.optimizers.Adam(learning_rate),
                  metrics=['accuracy'])

    return model

# callback for model to record accuracy
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.train_acc = []
        self.train_loss = []
        self.val_acc = []
        self.val_loss = []
        
    def on_epoch_end(self, batch, logs={}):
        self.train_acc.append(logs.get('accuracy'))
        self.train_loss.append(logs.get('loss'))
        self.val_acc.append(logs.get('val_accuracy'))
        self.val_loss.append(logs.get('val_loss'))

In [ ]:
encoder = LabelEncoder()
validation_encoded = encoder.fit_transform(validation)

In [ ]:
num = 20

patience = 10
layer_depths = [16, 32, 64]
kernel_sizes = [3, 3, 3]
dropout_rate = 0.5
learning_rate = 7*1e-5
num_classes = 16
output_prefix = f"EEG_epilepsy_{num}"
batch_size = 100
epochs = 1000
num_train = 16000

In [ ]:
y = keras.utils.to_categorical(validation_encoded, num_classes)
y_tr, y_vld = (y[:num_train], y[num_train:])

In [ ]:
X_tr, X_vld = (X[:num_train], X[num_train:])
y_tr, y_vld = (y[:num_train], y[num_train:])

In [ ]:
# model generation
history = AccuracyHistory()
earlystopper = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
model = cnn_model(layer_depths, kernel_sizes,
                  dropout_rate=dropout_rate, learning_rate=learning_rate,
                  num_classes=num_classes)

In [ ]:
# train!
model.fit(X_tr, y_tr,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_vld, y_vld),
          callbacks=[history, earlystopper])

training_score = model.evaluate(X_tr, y_tr, verbose=0)
validation_score = model.evaluate(X_vld, y_vld, verbose=0)

In [ ]:
# save model
model.save(f'my_model_{num}.keras')
output_prefix = f"EEG_epilepsy_{num}"
output_path = f"../results/{num}/"

In [ ]:
# save information and results

## parameters and final performance
params_file = open(output_path + f'{output_prefix}-params.txt','w')
        
params_file.write(f'Training Loss:       {training_score[0]}\n')
params_file.write(f'Training Accuracy:   {training_score[1]}\n')
params_file.write(f'Validation Loss:     {validation_score[0]}\n')
params_file.write(f'Validation Accuracy: {validation_score[1]}\n')
params_file.close()

## training, validation  loss, accuracy
tr_loss_file = open(output_path + f'{output_prefix}-tr-loss.txt', 'w')
tr_acc_file = open(output_path + f'{output_prefix}-tr-acc.txt', 'w')
val_loss_file = open(output_path + f'{output_prefix}-val-loss.txt', 'w')
val_acc_file = open(output_path + f'{output_prefix}-val-acc.txt', 'w')

for file_handler, training_data in zip([tr_loss_file, tr_acc_file,
                                        val_loss_file, val_acc_file],
                                       [history.train_loss, history.train_acc,
                                        history.val_loss, history.val_acc]):
    for index, item in enumerate(training_data):
        file_handler.write(f'{index}\t{item}\n')
    file_handler.close()

In [ ]:
# Графики

epochs = range(1, len(history.val_acc) + 1)

plt.figure(figsize=(12, 6), dpi=300)

# Validation Accuracy
plt.subplot(1, 2, 1)
plt.scatter(epochs, history.val_acc, linestyle='-', color='g', label='Validation Accuracy', s=5)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Validation Accuracy', fontsize=12)
plt.title('Validation Accuracy', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.ylim(0, 1)

# Training & Validation Loss
plt.subplot(1, 2, 2) 
plt.scatter(epochs, history.train_loss, linestyle='-', color='r', label='Training Loss', alpha=0.7, s=5)
plt.scatter(epochs, history.val_loss, linestyle='-', color='b', label='Validation Loss', alpha=0.7, s=5)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training & Validation Loss', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.ylim(bottom=0) 

plt.tight_layout()

# Сохранение
plt.savefig(output_path + f'metrics_plot_{num}.png', dpi=300, bbox_inches='tight')

plt.show()